# Geoapify Places 5 Cities

In [32]:
# Dependencies
from census import Census
import hvplot.pandas
import time
import requests
import json
import pandas as pd
import numpy as np
from scipy.stats import linregress
from matplotlib import pyplot as plt

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import the API key
from config_key import geoapify_key
from config_key import census_key

In [33]:
# Create an instance of the Census library
# 
c = Census(
    census_key, 
    year=2021
)

# Run Census Search to retrieve data on all zip codes (2021 ACS5 Census)
census_data = c.acs5.get(
    (
        "B01003_001E",
        "B17001_002E"
    ),
    {'for': 'zip code tabulation area:*'}
)

# Convert to DataFrame
census_df = pd.DataFrame(census_data)

# Column renaming
census_df = census_df.rename(
    columns = {
        "B01003_001E": "Population",
        "B17001_002E": "Poverty Count",
        "zip code tabulation area": "Zipcode",
        #rural vs urban,
        #latitude and latitude
    }
)

In [34]:
census_df

,Population,Poverty Count,Zipcode
0,17126.0,11302.0,00601
1,37895.0,17121.0,00602
2,49136.0,23617.0,00603
3,5751.0,3139.0,00606
4,26153.0,11640.0,00610
...,...,...,...
33769,13.0,0.0,99923
33770,917.0,182.0,99925
33771,1445.0,252.0,99926
33772,11.0,0.0,99927


In [35]:
#batch API call
#url = "https://api.geoapify.com/v1/batch?apiKey=geoapify_key
#headers = {"Content-Type": "application/json"}
#data = '{"api":"/v2/places","params":{},"inputs":[{"id":"optional-input-id1","params":{}}]}'
      
#try:
#    resp = requests.post(url, headers=headers, data=data)
#    print(resp.json())
#except requests.exceptions.HTTPError as e:
#    print (e.response.text)

In [36]:
cities = ["Bronx, NY","Napa, CA", "Victorville, CA","San Francisco, CA"]

places = ["512ffe11e13a7852c059ed90ac0d5f6c4440f00101f9010ce3930000000000c00205",
          "512f63af1c46925ec0597f784c9308264340f00101f90177b4010000000000c00208",
          "512461df4ea2525dc0594fd5f3249f444140f00101f901e0b4010000000000c00208",
          "513a596abddf9a5ec05913dd6921b7e34240f00101f90160b5010000000000c00208"]
rows = []
responses = []

In [37]:
for place in places:
    filters = f"place:{place}"
    categories = "commercial.supermarket"
    #radius = 16100 # 10 miles
    #bias

    params = {
        "categories":categories,
        "apiKey":geoapify_key,
        "filter":filters #,
    }

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url,params=params)

    # convert response to json
    supermarket_data = response.json()

    # print the response url, avoid doing for public github repos in order to avoid exposing key
    print(response.url)
    responses.append(supermarket_data)

https://api.geoapify.com/v2/places?categories=commercial.supermarket&apiKey=bf8e20276a9741e3b5a704ce3a508735&filter=place%3A512ffe11e13a7852c059ed90ac0d5f6c4440f00101f9010ce3930000000000c00205
https://api.geoapify.com/v2/places?categories=commercial.supermarket&apiKey=bf8e20276a9741e3b5a704ce3a508735&filter=place%3A512f63af1c46925ec0597f784c9308264340f00101f90177b4010000000000c00208
https://api.geoapify.com/v2/places?categories=commercial.supermarket&apiKey=bf8e20276a9741e3b5a704ce3a508735&filter=place%3A512461df4ea2525dc0594fd5f3249f444140f00101f901e0b4010000000000c00208
https://api.geoapify.com/v2/places?categories=commercial.supermarket&apiKey=bf8e20276a9741e3b5a704ce3a508735&filter=place%3A513a596abddf9a5ec05913dd6921b7e34240f00101f90160b5010000000000c00208


In [7]:
# r = []

for elephant in responses:
    # Print the json (pretty printed)
    # print(json.dumps(response, indent=4, sort_keys=True))

    # Extracting 'features' and 'properties' from JSON
    features = elephant.get("features", [])

    # Extracting results from JSON 
    # results = supermarket_data.get("results", []) # Create a list to store rows 

    for feature in features: 
        properties = feature.get("properties",{})
        # Extract relevant information
        name = properties.get("name") 
        lat = properties.get("lat") 
        lon = properties.get("lon") # Append data to the list
        postcode = properties.get("postcode")
        radius = properties.get("radius")
        city = properties.get("city")
        rows.append({"Name": name, "City": city, "Zipcode": postcode, "Latitude": lat, "Longitude": lon}) 
    print(len(rows))
#    r.append(rows)

20
32
42
62


In [10]:
#r
#len(r[3])

In [11]:
# Create a DataFrame 
supermarket_df = pd.DataFrame(rows) # Display the DataFrame 

supermarket_df["City"].value_counts()

City
New York         20
San Francisco    20
Napa             12
Victorville      10
Name: count, dtype: int64

In [12]:
supermarket_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Name       62 non-null     object 
 1   City       62 non-null     object 
 2   Zipcode    62 non-null     object 
 3   Latitude   62 non-null     float64
 4   Longitude  62 non-null     float64
dtypes: float64(2), object(3)
memory usage: 2.5+ KB


In [13]:
supermarket_df.columns

Index(['Name', 'City', 'Zipcode', 'Latitude', 'Longitude'], dtype='object')

In [14]:
supermarket_totals_df = supermarket_df.groupby(['City', 'Zipcode']).size().reset_index(name='Supermarket Count')
# supermarket_totals_df = pd.DataFrame[{"City":supermarket_df["City"],"Zipcode":supermarket_df["Zipcode"],"Supermarket Count":supermarket_df["Supermarket Count"]}].groupby["Zipcode"]
supermarket_totals_df

,City,Zipcode,Supermarket Count
0,Napa,94558,8
1,Napa,94559,4
2,New York,10451,4
3,New York,10455,1
4,New York,10458,1
5,New York,10460,1
6,New York,10461,3
7,New York,10463,4
8,New York,10467,1
9,New York,10468,1


In [15]:
census_df

,Population,Poverty Count,Zipcode
0,17126.0,11302.0,00601
1,37895.0,17121.0,00602
2,49136.0,23617.0,00603
3,5751.0,3139.0,00606
4,26153.0,11640.0,00610
...,...,...,...
33769,13.0,0.0,99923
33770,917.0,182.0,99925
33771,1445.0,252.0,99926
33772,11.0,0.0,99927


In [16]:
census_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33774 entries, 0 to 33773
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Population     33774 non-null  float64
 1   Poverty Count  33774 non-null  float64
 2   Zipcode        33774 non-null  object 
dtypes: float64(2), object(1)
memory usage: 791.7+ KB


In [17]:
# Merge datasets on Zipcodes
supermarkets_density_final = pd.merge(
    supermarket_totals_df,
    census_df,
    how = "left",
    on = ["Zipcode", "Zipcode"]
)
supermarkets_density_final

,City,Zipcode,Supermarket Count,Population,Poverty Count
0,Napa,94558,8,66345.0,4393.0
1,Napa,94559,4,27913.0,2704.0
2,New York,10451,4,50118.0,17073.0
3,New York,10455,1,42781.0,15120.0
4,New York,10458,1,86757.0,25752.0
5,New York,10460,1,59979.0,20684.0
6,New York,10461,3,52756.0,7641.0
7,New York,10463,4,73256.0,13353.0
8,New York,10467,1,102209.0,25754.0
9,New York,10468,1,82480.0,22073.0


In [18]:
#census_df[census_df['Zipcode']=='92392']

In [19]:
supermarkets_density_final = supermarkets_density_final.fillna(0)

In [20]:
supermarkets_density_final = supermarkets_density_final[["Population","Poverty Count"]].astype(int)
supermarkets_density_final

,Population,Poverty Count
0,66345,4393
1,27913,2704
2,50118,17073
3,42781,15120
4,86757,25752
5,59979,20684
6,52756,7641
7,73256,13353
8,102209,25754
9,82480,22073


In [21]:
# Add a Supermarket Count column (Supermarkets/Population)
#Supermarket_Total = supermarkets_density_final["Supermarket Count"].sum().astype(int)
#Population_Total = supermarkets_density_final["Population"].astype(int).groupby(["Zipcode"])
#Supermarket_Density = Supermarket_Total / Population_Total

# Configure the final DataFrame
#supermarkets_density_final = supermarkets_density_final[
#    [
#        "postcode",
#        "population",
#        "supermarket"
#    ]
#]

# Display DataFrame length and sample data
# print(f"Number of rows in the DataFrame: {len(supermarkets_density_final)}")
#supermarkets_density_final.head()

In [22]:
# Create DataFrame called US_locations_df to store the city, coordinates, supermarket property, distance
#https://api.geoapify.com/v2/places?categories=commercial.supermarket&filter=place:512ffe11e13a7852c059ed90ac0d5f6c4440f00101f9010ce3930000000000c00205&limit=100&apiKey=geoapify_key
#https://api.geoapify.com/v2/places?categories=commercial.supermarket&filter=place:512f63af1c46925ec0597f784c9308264340f00101f90177b4010000000000c00208&limit=100&apiKey=geoapify_key
#https://api.geoapify.com/v2/places?categories=commercial.supermarket&filter=place:512461df4ea2525dc0594fd5f3249f444140f00101f901e0b4010000000000c00208&limit=100&apiKey=geoapify_key
#https://api.geoapify.com/v2/places?categories=commercial.supermarket&filter=place:513a596abddf9a5ec05913dd6921b7e34240f00101f90160b5010000000000c00208&limit=100&apiKey=geoapify_key

cities = ["Bronx, NY","Napa, CA", "Victorville, CA","San Francisco, CA"]

places = ["512ffe11e13a7852c059ed90ac0d5f6c4440f00101f9010ce3930000000000c00205",
          "512f63af1c46925ec0597f784c9308264340f00101f90177b4010000000000c00208",
          "512461df4ea2525dc0594fd5f3249f444140f00101f901e0b4010000000000c00208",
          "513a596abddf9a5ec05913dd6921b7e34240f00101f90160b5010000000000c00208"]

place_Bronx = "512ffe11e13a7852c059ed90ac0d5f6c4440f00101f9010ce3930000000000c00205"
place_Napa = "512f63af1c46925ec0597f784c9308264340f00101f90177b4010000000000c00208"
place_Victorville = "512461df4ea2525dc0594fd5f3249f444140f00101f901e0b4010000000000c00208"
place_SanFran = "513a596abddf9a5ec05913dd6921b7e34240f00101f90160b5010000000000c00208"
        


# Set the parameters to search for supermarkets 

#latitude = [40.837048,38.297539,34.536217,37.828724,44.8485]
#longitude = [-73.865433,-122.286865,-117.292763,-122.355537,-123.2340]
# This radius is equivalent to 10 miles, which is the definition of food desert for rural cities
#radius = 16100 # 10 miles
#radius = 1610 # 1 mile
# Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
#filters = f"circle:{longitude},{latitude},{radius}"

filters = f"place:{places[2]}"
categories = "commercial.supermarket"
# This radius is equivalent to 10 miles, which is the definition of food desert for rural cities
radius = 16100 # 10 miles
#radius = 1610 # 1 mile
#bias = f"proximity:{longitude},{latitude}"
#limit = 100


# Set up a params dictionary
params = {
    "categories":categories,
    "apiKey":geoapify_key,
#    "limit": limit,
    "filter":filters #,
#    "bias":bias
}

# Set base URL
base_url = "https://api.geoapify.com/v2/places"

# Make an API request using the params dictionary
response = requests.get(base_url,params=params)

# print the response url, avoid doing for public github repos in order to avoid exposing key
print(response.url)    

https://api.geoapify.com/v2/places?categories=commercial.supermarket&apiKey=bf8e20276a9741e3b5a704ce3a508735&filter=place%3A512461df4ea2525dc0594fd5f3249f444140f00101f901e0b4010000000000c00208


In [23]:
#c.acs5.tables()

In [24]:
# Create DataFrame called US_locations_df to store the city, coordinates, supermarket property, distance
#https://api.geoapify.com/v2/places?categories=commercial.supermarket&filter=place:512ffe11e13a7852c059ed90ac0d5f6c4440f00101f9010ce3930000000000c00205&limit=100&apiKey=YOUR_API_KEY
#https://api.geoapify.com/v2/places?categories=commercial.supermarket&filter=place:512f63af1c46925ec0597f784c9308264340f00101f90177b4010000000000c00208&limit=100&apiKey=YOUR_API_KEY
#https://api.geoapify.com/v2/places?categories=commercial.supermarket&filter=place:512461df4ea2525dc0594fd5f3249f444140f00101f901e0b4010000000000c00208&limit=100&apiKey=YOUR_API_KEY
#https://api.geoapify.com/v2/places?categories=commercial.supermarket&filter=place:513a596abddf9a5ec05913dd6921b7e34240f00101f90160b5010000000000c00208&limit=100&apiKey=YOUR_API_KEY

cities = ["Bronx, NY","Napa, CA", "Victorville, CA","San Francisco, CA"]

places = ["512ffe11e13a7852c059ed90ac0d5f6c4440f00101f9010ce3930000000000c00205",
          "512f63af1c46925ec0597f784c9308264340f00101f90177b4010000000000c00208",
          "512461df4ea2525dc0594fd5f3249f444140f00101f901e0b4010000000000c00208",
          "513a596abddf9a5ec05913dd6921b7e34240f00101f90160b5010000000000c00208"]

place_Bronx = "512ffe11e13a7852c059ed90ac0d5f6c4440f00101f9010ce3930000000000c00205"
place_Napa = "512f63af1c46925ec0597f784c9308264340f00101f90177b4010000000000c00208"
place_Victorville = "512461df4ea2525dc0594fd5f3249f444140f00101f901e0b4010000000000c00208"
place_SanFran = "513a596abddf9a5ec05913dd6921b7e34240f00101f90160b5010000000000c00208"
          
#latitude = [40.837048,38.297539,34.536217,37.828724,44.8485]
#longitude = [-73.865433,-122.286865,-117.292763,-122.355537,-123.2340]


# Set the parameters to search for supermarkets 
categories = "commercial.supermarket"
# This radius is equivalent to 10 miles, which is the definition of food desert for rural cities
#radius = 16100 # 10 miles
#radius = 1610 # 1 mile
i = 0

# Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
#filters = f"circle:{longitude},{latitude},{radius}"

filters = f"place:{places[2]}"
categories = "commercial.supermarket"
# This radius is equivalent to 10 miles, which is the definition of food desert for rural cities
radius = 16100 # 10 miles
#radius = 1610 # 1 mile
#bias = f"proximity:{longitude},{latitude}"
#limit = 100


# Set up a params dictionary
params = {
    "categories":categories,
    "apiKey":geoapify_key,
#    "limit": limit,
    "filter":filters #,
#    "bias":bias
}

# Set base URL
base_url = "https://api.geoapify.com/v2/places"

# Make an API request using the params dictionaty
response = requests.get(base_url,params=params)

# print the response url, avoid doing for public github repos in order to avoid exposing key
print(response.url)    

https://api.geoapify.com/v2/places?categories=commercial.supermarket&apiKey=bf8e20276a9741e3b5a704ce3a508735&filter=place%3A512461df4ea2525dc0594fd5f3249f444140f00101f901e0b4010000000000c00208


In [25]:
    # convert response to json
    supermarket_data = response.json()

    # Print the json (pretty printed)
    print(json.dumps(supermarket_data, indent=4, sort_keys=True))

{
    "features": [
        {
            "geometry": {
                "coordinates": [
                    -117.39681949157831,
                    34.50927068192078
                ],
                "type": "Point"
            },
            "properties": {
                "address_line1": "Walmart Supercenter",
                "address_line2": "12234 Palmdale Road, Victorville, CA 92392, United States of America",
                "categories": [
                    "building",
                    "building.commercial",
                    "commercial",
                    "commercial.supermarket"
                ],
                "city": "Victorville",
                "country": "United States",
                "country_code": "us",
                "county": "San Bernardino County",
                "datasource": {
                    "attribution": "\u00a9 OpenStreetMap contributors",
                    "license": "Open Database Licence",
                    "raw": {
           